In [1]:
import transforms3d as tranf
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
import math
rng = np.random.default_rng(1337)

t = Time('2020-01-01T20:00:00')

location = EarthLocation(
    lon=-17.89 * u.deg, lat=28.76 * u.deg, height=2200 * u.m
)

# target horizontal coordinate frame
altazfunc = AltAz(obstime=t, location=location)
star = SkyCoord(alt=0 * u.deg, az=90 * u.deg, frame='altaz')

In [2]:
def altazunpack(altaz):
    return (altaz.az.rad, altaz.alt.rad)

def altaz2euler(altaz, roll=0):
    return (-altaz[0], altaz[1], roll)

def altaz2vec(altaz):
    return math.sin(altaz[0]) * math.cos(altaz[1]), math.cos(altaz[0]) * math.cos(altaz[1]), math.sin(altaz[1])


In [5]:
north = np.array((0, 1, 0))
east = np.array((1, 0, 0))
norm_ground = np.array((1, 0, 1))
norm_ground = norm_ground / np.linalg.norm(norm_ground)
new_north = np.cross(norm_ground, east)
new_north = new_north / np.linalg.norm(new_north)
new_east = np.cross(new_north, norm_ground)
transform = np.vstack((new_east, new_north, norm_ground))
transform_inv = np.linalg.inv(transform)
transform_q = tranf.quaternions.mat2quat(transform.T)
transform_q_inv = tranf.quaternions.qinverse(transform_q)

altaz = altazunpack(star)
star_vec = altaz2vec(altaz)
new_star_vec1 = star_vec@transform_inv
new_star_vec = tranf.quaternions.rotate_vector(star_vec, transform_q_inv)

new_baseline = np.dot(new_star_vec, [0,0,1])
baseline_azimuth = np.rad2deg(np.arctan2(new_star_vec[0], new_star_vec[1]))

In [6]:
print(f"Baseline azimuth: {baseline_azimuth:.10f} deg")

Baseline azimuth: 90.0000000000 deg
